In [24]:
import duckdb

In [30]:
with duckdb.connect("consumer/data/staging.db") as conn:

    conn.sql("SELECT * FROM orders").show()

┌──────────┬─────────────┬────────────────────────────┬─────────┬─────────────────┬─────────┬─────────┬───────────────┬───────────────┬───────────────────┬────────────────┬──────────┐
│ order_id │ customer_id │         order_date         │ status  │ shipping_method │  notes  │   op    │ emitted_ts_ms │     ts_ms     │ connector_version │ transaction_id │   lsn    │
│  int32   │    int32    │         timestamp          │ varchar │     varchar     │ varchar │ varchar │     int64     │     int64     │      varchar      │    varchar     │  int64   │
├──────────┼─────────────┼────────────────────────────┼─────────┼─────────────────┼─────────┼─────────┼───────────────┼───────────────┼───────────────────┼────────────────┼──────────┤
│        1 │           1 │ 2025-05-10 09:24:21.717759 │ pending │ Standard        │ NULL    │ c       │ 1746833062131 │ 1746833061725 │ 2.4.0.Final       │ 747            │ 24461720 │
│        2 │           2 │ 2025-05-10 09:24:21.717759 │ shipped │ Express       

In [5]:
conn.close()